In [1]:
#from rostering_solver import Instance, Solver
import pandas as pd
import numpy as np
import json
import random
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

# from solver import Instance
from solver_alberto import Instance as Instance_A, Solver as Solver_A

from argparse import Namespace
import sys
sys.path.append('../../solver')
from solver_shifts import run_solver_shift_return, Instance, Solver

In [2]:
# READ SHIFTS
city = 'frankfurt'
demand_baseline = "1.00"
week_demand_type = "doublepeak" # "fixed" demand_type
demand_type = "uniform"
outsourcing_cost_multiplier = 1.5
model = "fixed" 
OC = 1.5
RM = 1.5
GM = 0.8
max_n_shift = 100

key = f'{city}_db={demand_baseline}_dt={demand_type}'

shifts_file = f'/Users/jonnycodd/Documents/MASTERS/Master Thesis/lmd-sizing-scheduling/shifts/{city}_db={demand_baseline}_dt={demand_type}.json'
with open(shifts_file, 'r') as file:
    shifts_data = json.load(file)

shifts_df = (
    pd.DataFrame(shifts_data).query(f'outsourcing_cost_multiplier == {outsourcing_cost_multiplier} & demand_type == "{demand_type}" & model == "{model}"')
    .query('regional_multiplier == 1.5 & global_multiplier == 0.8')
)

shifts_df.head(10)

,instance_file,city,demand_baseline,demand_type,model,max_n_shifts,outsourcing_cost_multiplier,regional_multiplier,global_multiplier,region,shifts_start,shifts_end
20,frankfurt_db=1.00_dt=uniform.json,frankfurt,1.0,uniform,fixed,NaN,1.5,1.5,0.8,0,"{'0': 0, '1': 4}","{'0': 4, '1': 8}"
21,frankfurt_db=1.00_dt=uniform.json,frankfurt,1.0,uniform,fixed,NaN,1.5,1.5,0.8,1,"{'0': 0, '1': 4}","{'0': 4, '1': 8}"
22,frankfurt_db=1.00_dt=uniform.json,frankfurt,1.0,uniform,fixed,NaN,1.5,1.5,0.8,2,"{'0': 0, '1': 4}","{'0': 4, '1': 8}"
23,frankfurt_db=1.00_dt=uniform.json,frankfurt,1.0,uniform,fixed,NaN,1.5,1.5,0.8,3,"{'0': 0, '1': 4}","{'0': 4, '1': 8}"


# Clarices solver

In [3]:

instance_file = f'/Users/jonnycodd/Documents/MASTERS/Master Thesis/lmd-sizing-scheduling/instances/{city}_db={demand_baseline}_dt={demand_type}.json'
dict_shifts, n_regions, dict_raw = run_solver_shift_return(model=model, instance=instance_file , outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM, max_n_shifts=max_n_shift)

in instance
in solver
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-12
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7936 rows, 9840 columns and 16888 nonzeros
Model fingerprint: 0x81242d3b
Variable types: 7680 continuous, 2160 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+01]
Found heuristic solution: objective 823.3500000
Presolve removed 7680 rows and 8674 columns
Presolve time: 0.03s
Presolved: 256 rows, 1166 columns, 2520 nonzeros
Found heuristic solution: objective 564.8083403
Variable types: 324 continuous, 842 integer (0 binary)

Root relaxation: objective 3.059750e+02, 177 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bound

In [4]:
# 3.059750e+02
dict_shifts

{3: {'shifts_start': {0: 0, 1: 4}, 'shifts_end': {0: 4, 1: 8}},
 0: {'shifts_start': {0: 0, 1: 4}, 'shifts_end': {0: 4, 1: 8}},
 1: {'shifts_start': {0: 0, 1: 4}, 'shifts_end': {0: 4, 1: 8}},
 2: {'shifts_start': {0: 0, 1: 4}, 'shifts_end': {0: 4, 1: 8}}}

In [5]:
# Define the arguments as a dictionary
arguments = {
    'model': 'fixed', # choices=('base', 'fixed', 'partflex', 'flex')
    'instance': instance_file,
    'outsourcing_cost_multiplier': OC,
    'regional_multiplier': RM,
    'global_multiplier': GM,
    'max_n_shifts': 10,
    'output': 'output.txt'
}

# Create a Namespace object from the dictionary
args = Namespace(**arguments)

i = Instance(args=args)
solver = Solver(args=args, i=i)

solver.solve_fixed()

in instance
in solver
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7936 rows, 9840 columns and 16888 nonzeros
Model fingerprint: 0x81242d3b
Variable types: 7680 continuous, 2160 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+01]
Found heuristic solution: objective 823.3500000
Presolve removed 7680 rows and 8674 columns
Presolve time: 0.01s
Presolved: 256 rows, 1166 columns, 2520 nonzeros
Found heuristic solution: objective 564.8083403
Variable types: 324 continuous, 842 integer (0 binary)

Root relaxation: objective 3.059750e+02, 177 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

{'instance': 'frankfurt_db=1.00_dt=uniform',
 'model': 'fixed',
 'city': 'frankfurt',
 'DB': 1.0,
 'DT': 'uniform',
 'OC': 1.5,
 'RM': 1.5,
 'GM': 0.8,
 'num_periods': 8,
 'num_scenarios': 30,
 'obj_value': 305.97500900000085,
 'elapsed_time': 0.029708147048950195,
 'n_variables': 9840,
 'n_constraints': 7936,
 'n_nonzeroes': 16888,
 'courier_moved_pct': 3.125,
 'hiring_costs': 256.0,
 'outsourcing_costs': 49.97500900000085,
 'hired_couriers': {'60320': [1, 1, 1, 1, 1, 1, 1, 1],
  '60389': [1, 1, 1, 1, 1, 1, 1, 1],
  '60388': [1, 1, 1, 1, 1, 1, 1, 1],
  '60386': [1, 1, 1, 1, 1, 1, 1, 1],
  '60435': [1, 1, 1, 1, 2, 2, 2, 2],
  '60327': [1, 1, 1, 1, 1, 1, 1, 1],
  '60596': [1, 1, 1, 1, 1, 1, 1, 1],
  '60529': [1, 1, 1, 1, 1, 1, 1, 1],
  '60594': [1, 1, 1, 1, 1, 1, 1, 1],
  '60599': [1, 1, 1, 1, 1, 1, 1, 1],
  '60486': [1, 1, 1, 1, 1, 1, 1, 1],
  '60528': [1, 1, 1, 1, 1, 1, 1, 1],
  '60326': [1, 1, 1, 1, 1, 1, 1, 1],
  '60598': [1, 1, 1, 1, 1, 1, 1, 1],
  '60433': [1, 1, 1, 2, 1, 1, 1, 1]

# Albertos solver

In [17]:
city = 'berlin'
demand_baseline = "2.00"
week_demand_type = "doublepeak" # "fixed" demand_type
demand_type = "uniform"
outsourcing_cost_multiplier = 1.5
OC = 1.5
RM = 1.5
GM = 0.8
max_n_shift = 100
instance_file = f'/Users/jonnycodd/Documents/MASTERS/Master Thesis/lmd-sizing-scheduling/instances/{city}_db={demand_baseline}_dt={demand_type}.json'

In [20]:
# Define the arguments as a dictionary
# fixed = 3.05975, flex = 305.6
arguments = {
    'model': 'flex', # choices=('base', 'fixed', 'partflex', 'flex')
    'instance': instance_file,
    'outsourcing_cost_multiplier': OC,
    'regional_multiplier': RM,
    'global_multiplier': GM,
    'max_n_shifts': 10,
    'output': 'output.txt'
}

# Create a Namespace object from the dictionary
args = Namespace(**arguments)

i = Instance_A(args=args)
solver = Solver_A(args=args, i=i)

output = solver.solve_flex()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14692 rows, 22728 columns and 42846 nonzeros
Model fingerprint: 0xeb79070d
Variable types: 14160 continuous, 8568 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+02]
Found heuristic solution: objective 2881.6000000
Presolve removed 13751 rows and 14123 columns
Presolve time: 0.03s
Presolved: 941 rows, 8605 columns, 17105 nonzeros
Variable types: 1462 continuous, 7143 integer (0 binary)

Root relaxation: objective 8.400125e+02, 943 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     840.0125210  840.01252 